# Explore the results of the given examples

### Import the required packages and disable some expected warnings

In [1]:
import logging

from morphio import set_ignored_warning
from morphio import Warning
from neurom import load_morphology
from neurom.view import plotly_impl

logging.getLogger("neurots.generate.grower").disabled = True
set_ignored_warning([Warning.zero_diameter], True)

### Run the example extract_synthesis_inputs.py and plot the result

In [2]:
%run extract_synthesis_inputs.py

morph = load_morphology("results_extract_synthesis_inputs/generated_cell.swc")
_ = plotly_impl.plot_morph3d(morph, inline=True)

### Run the example synthesize_single_neuron.py and plot the result

In [3]:
%run synthesize_single_neuron.py

morph = load_morphology("results_single_neuron/generated_cell.swc")
_ = plotly_impl.plot_morph3d(morph, inline=True)

JSONDecodeError: Expecting ',' delimiter: line 339 column 1 (char 8011)

RawDataError: File: results_single_neuron/generated_cell.swc does not exist.

### Run the example synthesize_neurons.py and plot the result

In [ ]:
%run synthesize_neurons.py

for i in range(10):
    morph = load_morphology(f"results_neurons/generated_cell_{i}.swc")
    _ = plotly_impl.plot_morph3d(morph, inline=True)

### Run the example synthesize_neuron_with_diameters.py and plot the result

In [ ]:
%run synthesize_neuron_with_diameters.py

morph = load_morphology("results_neuron_with_diameters/generated_cell.swc")
_ = plotly_impl.plot_morph3d(morph, inline=True)

### Run the example synthesize_neuron_external_diameter.py and plot the result

In [ ]:
%run synthesize_neuron_external_diameter.py

morph = load_morphology("results_neuron_external_diameter/generated_cell.swc")
_ = plotly_impl.plot_morph3d(morph, inline=True)